### Домашнее задание <a class="anchor" id="hw"></a><center>

In [35]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

__Задача:__ подойдет ли репетитор для подготовки к экзамену по математике

In [36]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [37]:
# Объявляет функцию standard_scale, которая принимает один аргумент X.
# X обычно представляет собой матрицу признаков (набор данных),
# где строки — это наблюдения, а столбцы — это признаки (фичи).
def standard_scale(X):

    # Вычисляется среднее значение для каждого столбца в матрице X.
    # Параметр axis=0 указывает на то, что среднее значение вычисляется
    # вдоль столбцов (то есть для каждого признака отдельно).
    # Результат сохраняется в переменной mean, которая является массивом,
    # содержащим средние значения для каждого признака.
    mean = X.mean(axis=0)

    # Вычисляется стандартное отклонение для каждого столбца в матрице X.
    # Параметр axis=0 указывает на то, что
    # стандартное отклонение вычисляется вдоль столбцов.
    # Результат сохраняется в переменной std, которая является
    # массивом стандартных отклонений для каждого признака.
    std = X.std(axis=0)

    # Выполняет стандартизацию каждого признака в матрице X.
    # От каждого элемента столбца вычитается
    # соответствующее среднее значение (mean),
    # а затем результат делится на соответствующее стандартное отклонение (std).
    # Таким образом, каждый признак будет иметь
    # среднее значение 0 и стандартное отклонение 1.
    # Полученный массив возвращается в качестве результата функции.
    return (X - mean) / std

# Таким образом, вся функция standard_scale нормализует входную матрицу X,
# приводя каждый признак к нулевому среднему значению и единичному стандартному отклонению.
# Это часто используется в машинном обучении для улучшения производительности моделей.

In [38]:
# Создает копию матрицы X и преобразует все ее элементы в тип данных float64.
# Преобразование в float64 может быть полезным для обеспечения точности вычислений,
# особенно если исходные данные имеют другой числовой тип (например, int).
# Копия сохраняется в переменной X_st,
# чтобы изменения не затрагивали исходную матрицу X.
X_st = X.copy().astype(np.float64)

# Выполняет стандартизацию подмножества столбцов матрицы X_st,
# а именно столбцов с индексами от 1 до 3 включительно
# (в Python, как и во многих других языках программирования,
#  срезы массивов включают начальный индекс и исключают конечный). 
# Сначала извлекается подмножество матрицы X_st X_st[:, 1:4],
# которое передается в функцию standard_scale для стандартизации.
# Результат заменяет исходные значения в тех же столбцах X_st.
# - X_st[:, 1:4] выбирает все строки и столбцы с индексами 1, 2 и 3.
# - standard_scale(X_st[:, 1:4]) стандартизирует выбранные столбцы,
# возвращая новую матрицу, где каждое значение нормализовано.
# - X_st[:, 1:4] = ... обновляет эти столбцы
# в X_st стандартизированными значениями.
X_st[:, 1:4] = standard_scale(X_st[:, 1:4])

X_st

array([[ 1.        , -0.70710678, -0.97958969, -0.89625816],
       [ 1.        , -0.70710678, -0.56713087, -0.89625816],
       [ 1.        , -0.35355339, -0.46401617,  0.38411064],
       [ 1.        ,  0.70710678, -0.77336028, -0.89625816],
       [ 1.        ,  0.        ,  0.97958969,  0.38411064],
       [ 1.        , -1.06066017, -0.36090146, -0.89625816],
       [ 1.        ,  0.70710678,  1.08270439,  1.66447944],
       [ 1.        ,  2.47487373,  2.11385144,  1.66447944],
       [ 1.        , -0.70710678, -1.08270439, -0.89625816],
       [ 1.        , -0.35355339,  0.05155735,  0.38411064]])

**1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.**

1 метод

In [39]:
# Объявляет функцию calc_logloss, которая принимает два аргумента:
# - y: массив истинных меток (значений) целевой переменной.
# - y_pred: массив предсказанных вероятностей принадлежности к классу 1.
def calc_logloss(y, y_pred):

    # Заменяет значения y_pred, равные 1, на немного меньшее значение (1 - 1e-8).
    # Это необходимо, чтобы избежать логарифма от нуля при вычислении логарифмической функции потерь,
    # поскольку log(1 - y_pred) при y_pred = 1 будет равен log(0), что ведет к бесконечности.
    # Здесь 1e-8 — это небольшая константа, которая предотвращает эту проблему.
    y_pred = np.where(y_pred == 1, 1 - 1e-8, y_pred)

    # Аналогично, эта строка заменяет значения y_pred, равные 0,
    # на немного большее значение (0 + 1e-8).
    # Это делается для избежания вычисления log(0) при вычислении
    # логарифмической функции потерь, что также ведет к бесконечности.
    y_pred = np.where(y_pred == 0, 0 + 1e-8, y_pred)

    # Вычисляет логарифмическую потерю (LogLoss). 
    # - y * np.log(y_pred) вычисляет вклад положительных примеров в LogLoss.
    # - (1.0 - y) * np.log(1.0 - y_pred) вычисляет вклад отрицательных примеров в LogLoss.
    # - np.mean(...) вычисляет среднее значение по всем наблюдениям,
    # что соответствует делению суммы на количество наблюдений.
    # Знак минус - используется, чтобы перевести логарифмическую функцию потерь
    # из логарифмов вероятностей в положительную величину,
    # так как логарифмы вероятностей дают отрицательные значения.
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))

    # Возвращает вычисленное значение логарифмической потери err,
    # которое является средним логарифмической потери по всем наблюдениям.
    return err

# В итоге, функция calc_logloss принимает истинные метки и предсказанные вероятности,
# корректирует крайние значения предсказанных вероятностей,
# вычисляет логарифмическую потерю и возвращает её.

In [40]:
# Определяет небольшую константу eps.
# Эта константа будет использоваться для ограничения (clipping) значений,
# чтобы избежать проблем с логарифмическими вычислениями (например, логарифма нуля).
eps = 1e-8

# Использует функцию np.clip для ограничения значений
# в массиве [1, 0.5, 0] между eps и 1 - eps.
# 1. np.clip — это функция из библиотеки NumPy, которая ограничивает значения в массиве.
# Она заменяет значения, выходящие за указанные пределы, на сами эти пределы.
# 2. [1, 0.5, 0] — это входной массив, значения которого нужно ограничить.
# 3. a_min=eps — это минимальное значение, которое будет использоваться для ограничения.
# Любое значение в массиве, меньшее eps, будет заменено на eps.
# 4. a_max=1 - eps — это максимальное значение, которое будет использоваться для ограничения.
# Любое значение в массиве, большее 1 - eps, будет заменено на 1 - eps.
# Функция np.clip применяет эти ограничения ко всем элементам массива.
# - Значение 1 больше чем 1 - eps, поэтому оно будет заменено на 1 - eps.
# - Значение 0.5 уже находится в пределах от eps до 1 - eps, поэтому оно останется без изменений.
# - Значение 0 меньше чем eps, поэтому оно будет заменено на eps.
# Результатом выполнения np.clip([1, 0.5, 0], a_min=eps, a_max=1 - eps) будет:
# array([1 - 1e-8, 0.5, 1e-8])
# Таким образом, конечный результат будет:
# array([0.99999999, 0.5, 0.00000001])
# Цель этой операции — избежать проблем при вычислении логарифмов вероятностей в таких функциях,
# как логарифмическая потеря (LogLoss), где логарифм от 0 или 1
# приводит к бесконечности или неопределенным значениям.
np.clip([1, 0.5, 0], a_min=eps, a_max=1 - eps)

array([9.9999999e-01, 5.0000000e-01, 1.0000000e-08])

In [41]:
# Объявляет функцию calc_logloss, которая принимает два аргумента:
# - y: массив истинных меток (значений) целевой переменной.
# - y_pred: массив предсказанных вероятностей принадлежности к классу 1.
def calc_logloss(y, y_pred):

    # Определяет небольшую константу eps.
    # Эта константа будет использоваться для ограничения значений предсказанных вероятностей,
    # чтобы избежать логарифма нуля и бесконечных значений при вычислении логарифмов.
    eps = 1e-8

    # Использует функцию np.clip из библиотеки NumPy для ограничения
    # значений массива y_pred между eps и 1 - eps.
    # Функция np.clip заменяет значения в массиве,
    # выходящие за указанные пределы, на сами эти пределы:
    # - Любое значение в y_pred, меньшее eps, будет заменено на eps.
    # - Любое значение в y_pred, большее 1 - eps, будет заменено на 1 - eps.
    # Это необходимо, чтобы избежать вычислений вида log(0) и log(1) в следующих строках,
    # которые приводят к неопределенным значениям или бесконечности.
    y_pred = np.clip(y_pred, eps, 1 - eps)

    # Вычисляет логарифмическую потерю (LogLoss).
    # - y * np.log(y_pred) вычисляет вклад положительных примеров в LogLoss.
    # - (1.0 - y) * np.log(1.0 - y_pred) вычисляет вклад отрицательных примеров в LogLoss.
    # - np.mean(...) вычисляет среднее значение по всем наблюдениям,
    # что соответствует делению суммы на количество наблюдений.
    # Знак минус - используется для перевода логарифмической функции потерь
    # из логарифмов вероятностей в положительную величину,
    # так как логарифмы вероятностей дают отрицательные значения.
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))

    # Возвращает вычисленное значение логарифмической потери err,
    # которое является средним логарифмической потери по всем наблюдениям.
    return err

# В итоге, функция calc_logloss принимает истинные метки и предсказанные вероятности,
# ограничивает предсказанные вероятности в диапазоне от eps до 1 - eps,
# вычисляет логарифмическую потерю и возвращает её.

2 метод

In [42]:
# Код реализует функцию для вычисления логарифмической потери (LogLoss)
# без использования библиотечных функций, таких как np.clip или np.mean. 

# Определяет функцию calc_logloss, которая принимает два аргумента:
# - y: массив истинных меток (значений) целевой переменной.
# - y_pred: массив предсказанных вероятностей принадлежности к классу 1.
def calc_logloss(y, y_pred):

    # Инициализирует переменную err с нулевым значением.
    # В этой переменной будет накапливаться суммарная потеря для всех наблюдений.
    err = 0

    # Начинает цикл, который будет выполнен для каждого индекса i в диапазоне длины массива y.
    # Это позволяет перебирать каждую пару истинной метки и предсказанной вероятности.
    for i in range(len(y)):

        # Если предсказанная вероятность y_pred[i] равна нулю,
        # то мы используем формулу для логарифмической потери для случая,
        # когда истинное значение y[i] равно 1.
        # В этом случае потеря будет равна (1 - y[i]) * log(1 - y_pred[i]).
        # Заметим, что мы используем значение 1.0 - y_pred[i],
        # чтобы избежать вычисления логарифма отрицательного числа.
        if y_pred[i] == 0:
            err += (1.0 - y[i]) * np.log(1.0 - y_pred[i])

        # Если предсказанная вероятность y_pred[i] равна единице,
        # то мы используем формулу для логарифмической потери для случая,
        # когда истинное значение y[i] равно 1.
        # В этом случае потеря будет равна y[i] * log(y_pred[i]).
        elif y_pred[i] == 1:
            err += y[i] * np.log(y_pred[i])

        # В остальных случаях (когда предсказанная вероятность не равна ни нулю, ни единице),
        # мы используем полную формулу для логарифмической потери.
        # Она включает два слагаемых: одно для случая, когда истинное значение y[i] равно 1,
        # и второе для случая, когда y[i] равно 0.
        else:
            err += y[i] * np.log(y_pred[i]) + (1.0 - y[i]) * np.log(1.0 - y_pred[i])

    # Возвращает усредненное значение потери, деленное на количество наблюдений и знак минус,
    # чтобы вернуть результат в соответствии с общепринятым определением LogLoss. 
    return - err / len(y)

# Таким образом, функция calc_logloss вычисляет логарифмическую потерю,
# проходя по каждому наблюдению и используя соответствующую формулу
# в зависимости от предсказанной вероятности и истинного значения.

In [43]:
# Код использует ранее определенную функцию calc_logloss
# для вычисления логарифмической потери (LogLoss)
# для заданных истинных меток y1 и предсказанных вероятностей y_pred1.

# Создает массив y1, который содержит истинные метки для двух наблюдений.
# В данном случае первое наблюдение имеет метку 1 (принадлежит классу 1),
# а второе наблюдение имеет метку 0 (принадлежит классу 0).
y1 = np.array([1, 0])

# Создает массив y_pred1, который содержит предсказанные вероятности
# принадлежности к классу 1 для двух наблюдений.
# В данном случае первое наблюдение имеет предсказанную вероятность 0.9,
# а второе наблюдение имеет предсказанную вероятность 0.
y_pred1 = np.array([0.9, 0])

# Вызывает ранее определенную функцию calc_logloss и передает ей
# истинные метки y1 и предсказанные вероятности y_pred1.
# Функция calc_logloss вычислит логарифмическую потерю для этих данных.
calc_logloss(y1, y_pred1)

0.05268025782891314

In [44]:
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05268025782891314

In [45]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss(y1, y_pred1)

0.10536051565782628

**2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.**

In [46]:
# Код определяет функцию сигмоиды, которая используется в логистической регрессии
# для преобразования линейной комбинации весов
# и признаков в вероятность принадлежности к классу 1.

# Определяет функцию sigmoid с одним аргументом z,
# который представляет собой линейную комбинацию весов и признаков.
def sigmoid(z):

    # Вычисляет сигмоиду от аргумента z.
    # np.exp(-z) вычисляет экспоненту от отрицания z,
    # а 1 / (1 + ...) вычисляет сигмоиду по формуле.
    res = 1 / (1 + np.exp(-z))

    # Возвращает результат вычисления сигмоиды.
    # Результат представляет собой вероятность принадлежности
    # к классу 1 для данного наблюдения, представленную в диапазоне от 0 до 1.
    return res

# Итак, функция sigmoid принимает в качестве входных данных
# линейную комбинацию весов и признаков
# и возвращает вероятность принадлежности к классу 1 для данного наблюдения,
# преобразованную с использованием сигмоидной функции.

In [47]:
# Код реализует процесс обучения модели логистической регрессии с использованием градиентного спуска.

# Определяет функцию eval_model, которая принимает четыре аргумента:
# - X: матрица признаков.
# - y: вектор истинных меток классов.
# - iterations: количество итераций обучения.
# - eta: скорость обучения (по умолчанию 1e-4).
def eval_model(X, y, iterations, eta=1e-4):

    # Устанавливает начальное значение генератора
    # случайных чисел для воспроизводимости результатов.
    np.random.seed(42)

    # Инициализирует веса модели случайными значениями
    # из стандартного нормального распределения.
    # Размерность вектора весов соответствует количеству признаков в матрице X.
    W = np.random.randn(X.shape[1])

    # Вычисляет количество наблюдений в матрице признаков X.
    n = X.shape[0]
    
    # Начинает цикл, который будет повторен iterations раз
    # для обновления параметров модели.
    for i in range(iterations):

        # Вычисляют прогнозы модели, значение функции потерь (LogLoss)
        # и ошибку для текущих весов модели.
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        # Вычисляет градиент функции потерь по параметрам модели.
        # Градиент выражается как среднее значение градиентов для всех наблюдений.
        dQ = 1/n * X.T @ (y_pred - y)

        # Обновляет веса модели в направлении, противоположном градиенту,
        # с учетом скорости обучения eta.
        W -= eta * dQ

        # Выводит текущее значение итерации, веса модели и значение функции потерь
        # каждые 10% от общего количества итераций.
        if i % (iterations / 10) == 0:
            print(i, W, err)
    
    # Вычисляют и возвращают значение функции потерь
    # для финальных предсказаний и веса модели после завершения всех итераций.
    final_error = calc_logloss(y, y_pred)
    return W, final_error

1 метод

аналог GridSearch

In [48]:
from tqdm import tqdm

In [49]:
np.logspace(2, 4, 4, dtype=int), np.linspace(1e-2, 5, 10)

# Этот код использует функции np.logspace и np.linspace
# из библиотеки NumPy для создания массивов чисел с различными интервалами.

# np.logspace(2, 4, 4, dtype=np.int)
# Эта строка создает массив чисел, равномерно распределенных
# в логарифмической шкале от \(10^2\) до \(10^4\).
# В качестве аргументов передаются:
# - 2: начальное значение логарифма.
# - 4: конечное значение логарифма.
# - 4: количество чисел в массиве.
# - dtype=np.int: тип данных элементов массива, в данном случае целые числа.
# Таким образом, массив будет содержать четыре целых числа,
# равномерно распределенных в логарифмической шкале от 100 до 10000.

# np.linspace(1e-2, 5, 10)
# Эта строка создает массив чисел, равномерно распределенных в линейной шкале от \(10^{-2}\) до 5. В качестве аргументов передаются:
# - 1e-2: начальное значение.
# - 5: конечное значение.
# - 10: количество чисел в массиве.
# Таким образом, массив будет содержать десять чисел,
# равномерно распределенных в линейной шкале от 0.01 до 5. 

# В обоих случаях np.logspace и np.linspace создают массивы значений
# с различными интервалами для использования в различных вычислениях или визуализациях.

(array([  100,   464,  2154, 10000]),
 array([0.01      , 0.56444444, 1.11888889, 1.67333333, 2.22777778,
        2.78222222, 3.33666667, 3.89111111, 4.44555556, 5.        ]))

In [50]:
# Создает массив iterations, содержащий четыре значения,
# равномерно распределенные в логарифмической шкале от \(10^2\) до \(10^4\).
# Тип данных указан как целочисленный (dtype=np.int).
iterations = np.logspace(2, 4, 4, dtype=int)

# Создает массив etas, содержащий десять значений,
# равномерно распределенных в линейной шкале от \(10^{-2}\) до 5.
etas = np.linspace(1e-2, 5, 10)

# Инициализируют переменные best_error и best_params.
# best_error устанавливается в бесконечность (np.inf), а best_params в пустой словарь.
# Они будут использоваться для хранения лучшего значения ошибки
# и соответствующих параметров обучения.
best_error = np.inf
best_params = {}

# Запускают вложенный цикл для перебора всех комбинаций значений iterations и etas.
# Для каждой комбинации запускается обучение модели с использованием функции eval_model.
# Если текущее значение ошибки меньше best_error,
# то best_error обновляется на текущее значение ошибки,
# а best_params устанавливается в текущие параметры обучения (iteration и eta).
for iteration in tqdm(iterations):
    for eta in etas:
        W, error = eval_model(X_st, y, iterations=iteration, eta=eta)
        if error < best_error:
            best_error = error
            best_params = {
                'iteration': iteration,
                'eta': eta
            }

# Выводит информацию о лучшем значении ошибки и соответствующих параметрах обучения.
print(f'Ошибка {best_error} при параметрах {best_params}')

 25%|██▌       | 1/4 [00:00<00:00,  5.29it/s]

0 [ 0.49635916 -0.1409725   0.64535427  1.52178506] 0.760958797591889
10 [ 0.49265881 -0.16787071  0.62221328  1.50954192] 0.7467006818024327
20 [ 0.48868992 -0.19442917  0.59944411  1.49767566] 0.7328651153682048
30 [ 0.48446029 -0.22063955  0.57705409  1.48619252] 0.7194524935754576
40 [ 0.47997869 -0.24649338  0.55505065  1.4750986 ] 0.7064630339679188
50 [ 0.47525478 -0.27198192  0.53344129  1.46439984] 0.6938967869573868
60 [ 0.47029912 -0.29709626  0.51223358  1.45410204] 0.6817536325961876
70 [ 0.46512312 -0.32182725  0.49143512  1.44421083] 0.6700332616838687
80 [ 0.45973898 -0.34616556  0.47105359  1.4347317 ] 0.658735139989009
90 [ 0.45415964 -0.37010173  0.4510966   1.42566993] 0.6478584551577349
0 [ 0.47667685 -0.29112687  0.51593213  1.45276794] 0.760958797591889
10 [ 0.15302827 -1.07349673 -0.05330995  1.447242  ] 0.458878668542744
20 [ 0.0338664  -1.33389885 -0.1641304   1.79467485] 0.41846490870843844
30 [-2.07371606e-03 -1.53135455e+00 -2.56777284e-01  2.10047940e+00] 

 50%|█████     | 2/4 [00:00<00:01,  1.85it/s]

0 [ 0.33890064 -1.3422075  -0.39002287  0.96964809] 0.760958797591889
0 [ 0.31921832 -1.49236188 -0.51944502  0.90063097] 0.760958797591889
0 [ 0.49635916 -0.1409725   0.64535427  1.52178506] 0.760958797591889
0 [ 0.47667685 -0.29112687  0.51593213  1.45276794] 0.760958797591889
0 [ 0.45699453 -0.44128125  0.38650998  1.38375082] 0.760958797591889
0 [ 0.43731221 -0.59143562  0.25708784  1.3147337 ] 0.760958797591889
0 [ 0.4176299  -0.74159     0.1276657   1.24571658] 0.760958797591889
0 [ 0.39794758 -0.89174437 -0.00175644  1.17669946] 0.760958797591889
0 [ 0.37826527 -1.04189875 -0.13117859  1.10768234] 0.760958797591889
0 [ 0.35858295 -1.19205313 -0.26060073  1.03866522] 0.760958797591889
0 [ 0.33890064 -1.3422075  -0.39002287  0.96964809] 0.760958797591889
0 [ 0.31921832 -1.49236188 -0.51944502  0.90063097] 0.760958797591889


 75%|███████▌  | 3/4 [00:03<00:01,  1.64s/it]

0 [ 0.49635916 -0.1409725   0.64535427  1.52178506] 0.760958797591889
1000 [ 0.06528341 -1.25315687 -0.13031431  1.68209805] 0.4264242191110676
2000 [-0.00591593 -1.60120112 -0.2960273   2.21707777] 0.3819609701086998
3000 [-0.01097467 -1.85119187 -0.44498757  2.64578105] 0.35499278688362434
4000 [-2.64430856e-04 -2.04430387e+00 -5.82222091e-01  3.00431189e+00] 0.33646376256331906
5000 [ 0.01719932 -2.20052369 -0.70941905  3.31414696] 0.32274977674382266
6000 [ 0.03850056 -2.33129977 -0.82799326  3.58846186] 0.3120492881925757
7000 [ 0.06216276 -2.44374222 -0.93914508  3.83587188] 0.30336396882449485
8000 [ 0.08727955 -2.54253835 -1.04387339  4.06228822] 0.29609630457253083
9000 [ 0.11325547 -2.63091817 -1.14300269  4.27192266] 0.28986691616198346
0 [ 0.47667685 -0.29112687  0.51593213  1.45276794] 0.760958797591889
1000 [ 1.09439377 -4.68780498 -3.75952282  9.668002  ] 0.19782429297287787
2000 [ 1.8020679  -6.21006696 -5.53005669 13.51756994] 0.16059718650418514
3000 [ 2.29969558 -7.3

100%|██████████| 4/4 [00:15<00:00,  3.97s/it]

9000 [  9.96965864 -24.01983811 -31.78454258  66.61013199] 0.03701577777302837
Ошибка 0.03424062515544972 при параметрах {'iteration': 10000, 'eta': 5.0}


2 метод

Вручную

In [51]:
W, error = eval_model(X_st, y, iterations=100, eta=1)
# 1. `eval_model: Это функция, которая выполняет обучение модели
# логистической регрессии с помощью градиентного спуска.
# 2. X_st: Это матрица признаков, которая представляет собой набор данных,
# подготовленных для обучения модели. Обычно она предварительно масштабирована
# с помощью стандартизации или нормализации.
# 3. y: Это вектор истинных меток классов, соответствующих данным в X_st.
# 4. iterations=100: Это количество итераций обучения модели.
# В данном случае модель будет обучаться в течение 100 итераций.
# 5. eta=1: Это скорость обучения (learning rate),
# которая определяет шаг градиентного спуска.
# Значение 1 указывает на то, что на каждой итерации веса модели
# будут обновляться на величину, пропорциональную градиенту функции потерь.
# 6. W: Это вектор весов модели после обучения.
# 7. error`: Это значение функции потерь (в данном случае, LogLoss) на последней итерации обучения.
# Таким образом, данная строка кода запускает процесс обучения модели логистической регрессии
# с фиксированными параметрами: 100 итераций обучения и скорость обучения, равную 1.

0 [ 0.46121499 -0.40908382  0.41426183  1.39855008] 0.760958797591889
10 [ 0.0387634  -1.30386446 -0.15001914  1.74758184] 0.4254600283700163
20 [-0.01161527 -1.64023671 -0.31196552  2.27533881] 0.38108858271134755
30 [-0.01156172 -1.88360488 -0.46035705  2.69723081] 0.35422352980840865
40 [ 9.33444488e-04 -2.07189803e+00 -5.97280973e-01  3.05064764e+00] 0.3358002141655348
50 [ 0.01934395 -2.22448855 -0.72422385  3.35653644] 0.32217454105883114
60 [ 0.04127542 -2.35245169 -0.84255667  3.6277315 ] 0.3115442516500474
70 [ 0.06539266 -2.46266676 -0.95347146  3.87262358] 0.30291439965765515
80 [ 0.090846   -2.55967079 -1.05796576  4.09697143] 0.29569092767181626
90 [ 0.11707191 -2.64659092 -1.15686395  4.30488038] 0.28949721467181433


**3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).**

In [52]:
# Определяет функцию с именем calc_pred_proba, которая принимает два аргумента:
#    - W: это вектор весов модели логистической регрессии.
#    - X: это матрица признаков, для которой мы хотим вычислить предсказанные вероятности.
def calc_pred_proba(W, X):

    # Вычисляет предсказанные вероятности принадлежности к классу 1
    # для каждого наблюдения в матрице `X`.
    # Для этого сначала выполняется линейная комбинация весов модели и признаков
    # (произведение матриц `X` и `W` с помощью `np.dot`),
    # а затем результат подается на вход функции `sigmoid`,
    # чтобы преобразовать его в вероятность.
    y_pred_proba = sigmoid(np.dot(X, W))

    # возвращает вычисленные предсказанные вероятности принадлежности к классу 1. 
    return y_pred_proba

# Итак, функция `calc_pred_proba` принимает на вход
# вектор весов модели `W` и матрицу признаков `X`,
# а затем возвращает предсказанные вероятности
# принадлежности каждого наблюдения к классу 1.

In [53]:
# Код вычисляет предсказанные вероятности принадлежности к классу 1
# для набора данных X_st с использованием весов модели W.
calc_pred_proba(W, X_st)

array([0.32380055, 0.22296142, 0.96767742, 0.00787362, 0.65604029,
       0.36735518, 0.98703363, 0.14768606, 0.35244079, 0.94041789])

**4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).**

In [54]:
# Определяет функцию с именем calc_pred, которая принимает два аргумента:
#    - W: это вектор весов модели логистической регрессии.
#    - X: это матрица признаков, для которой мы хотим сделать предсказания.
def calc_pred(W, X):

    # Вызывает ранее определенную функцию `calc_pred_proba`
    # для вычисления предсказанных вероятностей принадлежности к классу 1
    # для каждого наблюдения в матрице `X`.
    # Результат сохраняется в переменной `y_pred_proba`.
    y_pred_proba = calc_pred_proba(W, X)

    # Создает вектор предсказанных меток классов на основе предсказанных вероятностей.
    # Если вероятность принадлежности к классу 1 больше 0.5,
    # то соответствующая метка класса будет 1, иначе - 0.
    # Функция `np.where` выполняет это условие
    # для каждого элемента вектора `y_pred_proba`.
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)

    # Возвращает вектор предсказанных меток классов.
    return y_pred

# Итак, функция `calc_pred` принимает на вход вектор весов модели `W` и матрицу признаков `X`,
# вычисляет предсказанные вероятности принадлежности к классу 1 для каждого наблюдения
# и на их основе делает предсказания меток классов, которые затем возвращает.

In [55]:
# Вызывает функцию calc_pred с двумя аргументами:
#    - W: это вектор весов модели логистической регрессии.
#    - X_st: это матрица признаков, для которой мы хотим сделать предсказания.
pred = calc_pred(W, X_st)

# Эта переменная будет содержать предсказанные метки классов,
# которые возвращаются из функции calc_pred.
pred

array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1])

**5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.**

*Accuracy*

In [56]:
# Определяет функцию с именем accuracy, которая принимает два аргумента:
#    - y: вектор истинных меток классов.
#    - y_pred: вектор предсказанных меток классов.
def accuracy(y, y_pred):

    # Вычисляет точность предсказаний, сравнивая вектор истинных меток `y`
    # с вектором предсказанных меток `y_pred`.
    # Она возвращает долю правильно предсказанных меток, то есть долю элементов вектора `y`,
    # которые равны соответствующим элементам вектора `y_pred`.
    # Функция `np.mean` вычисляет среднее значение булевого вектора,
    # преобразуя `True` в 1 и `False` в 0.
    accuracy = np.mean(y == y_pred)

    return accuracy

# Итак, функция `accuracy` принимает на вход два вектора меток классов -
# истинных `y` и предсказанных `y_pred`,
# и возвращает долю правильно предсказанных меток.

In [57]:
print(f'True \t  {y}')
print(f'Predicted {pred}')

True 	  [0 0 1 0 1 0 1 0 1 1]
Predicted [0 0 1 0 1 0 1 0 0 1]


In [58]:
accuracy(y, pred)

0.9

*Confusion Matrix*

In [59]:
# TP  FP
# FN  TN

In [60]:
# Определяет функцию с именем confusion_matrix, которая принимает два аргумента:
#    - y: вектор истинных меток классов.
#    - y_pred: вектор предсказанных меток классов.
def confusion_matrix(y, y_pred):

    # Создает нулевую матрицу размером 2x2,
    # которая будет представлять собой матрицу ошибок (confusion matrix)
    # для двух классов: класс 1 и класс 0.
    cm = np.zeros((2, 2))

    # Начинает цикл по всем элементам вектора истинных меток `y`.
    for i in range(len(y)): 

        # Если текущий элемент вектора истинных меток `y`
        # и вектора предсказанных меток `y_pred` равен 1 (означает истинно положительный случай),
        # то это увеличивает количество истинно положительных (True Positive) случаев в матрице ошибок на 1.
        if y[i] == y_pred[i] == 1: # TP
            cm[0][0] += 1

        # Если текущий элемент вектора истинных меток `y`
        # и вектора предсказанных меток `y_pred` равен 0 (означает истинно отрицательный случай),
        # то это увеличивает количество истинно отрицательных (True Negative) случаев в матрице ошибок на 1.
        elif y[i] == y_pred[i] == 0: # TN
            cm[1][1] += 1

        # Если истинная метка класса равна 1, а предсказанная метка класса не равна 1
        # (означает ложно отрицательный случай),
        # то это увеличивает количество ложно отрицательных (False Negative) случаев в матрице ошибок на 1.
        elif y[i] != y_pred[i] and y[i] == 1: # FN
            cm[1][0] += 1

        # Если истинная метка класса равна 0, а предсказанная метка класса не равна 0
        # (означает ложно положительный случай),
        # то это увеличивает количество ложно положительных (False Positive) случаев в матрице ошибок на 1.
        elif y[i] != y_pred[i] and y[i] == 0: # FP
            cm[0][1] += 1

    # Возвращает заполненную матрицу ошибок.
    return cm

# Итак, функция `confusion_matrix` принимает на вход вектор истинных меток классов `y`
# и вектор предсказанных меток классов `y_pred`, вычисляет и возвращает матрицу ошибок,
# которая содержит информацию о количестве верно и ошибочно классифицированных наблюдений для каждого класса.

In [61]:
cm = confusion_matrix(y, pred)
cm

array([[4., 0.],
       [1., 5.]])

*Precision*

In [62]:
# Определяет функцию с именем precision, которая принимает два аргумента:
#    - y: вектор истинных меток классов.
#    - y_pred: вектор предсказанных меток классов.
def precision(y, y_pred):

    # Вызывает функцию `confusion_matrix` для вычисления матрицы ошибок
    # на основе векторов истинных и предсказанных меток классов.
    cm = confusion_matrix(y, y_pred)

    # Эти строки извлекают из матрицы ошибок количество
    # истинно положительных (`TP`), истинно отрицательных (`TN`),
    # ложно положительных (`FP`) и ложно отрицательных (`FN`)
    # случаев соответственно.
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]

    # Вычисляет точность (precision) модели как отношение
    # истинно положительных случаев к сумме
    # истинно положительных и ложно положительных случаев.
    # Точность характеризует способность модели
    # классифицировать объекты положительного класса корректно.
    precision = TP / (TP + FP)

    # Возвращает вычисленное значение точности.
    return precision

# Итак, функция `precision` принимает на вход векторы истинных
# и предсказанных меток классов, вычисляет и возвращает точность модели.

In [63]:
precision(y, pred)

1.0

*Recall*

In [64]:
# Определяет функцию с именем recall, которая принимает два аргумента:
#    - y: вектор истинных меток классов.
#    - y_pred: вектор предсказанных меток классов.
def recall(y, y_pred):

    # Вызывает функцию `confusion_matrix` для вычисления матрицы ошибок
    # на основе векторов истинных и предсказанных меток классов.
    cm = confusion_matrix(y, y_pred)

    # Эти строки извлекают из матрицы ошибок количество
    # истинно положительных (`TP`), истинно отрицательных (`TN`),
    # ложно положительных (`FP`) и ложно отрицательных (`FN`)
    # случаев соответственно.
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]

    # Вычисляет полноту (recall) модели как отношение истинно положительных случаев
    # к сумме истинно положительных и ложно отрицательных случаев.
    # Полнота характеризует способность модели обнаруживать
    # все объекты положительного класса.
    recall = TP / (TP + FN)

    # Возвращает вычисленное значение полноты.
    return recall

# Итак, функция `recall` принимает на вход векторы истинных
# и предсказанных меток классов, вычисляет и возвращает полноту модели.

In [65]:
recall(y, pred)

0.8

*F-score*

In [66]:
# Определяет функцию с именем f_score, которая принимает два аргумента:
#    - y: вектор истинных меток классов.
#    - y_pred: вектор предсказанных меток классов.
def f_score(y, y_pred):

    # Вызывает функцию `precision` для вычисления точности модели.
    pr = precision(y, y_pred)

    # Вызывает функцию `recall` для вычисления полноты модели.
    rec = recall(y, y_pred)

    # Вычисляет F-меру (F-score) модели, которая является
    # гармоническим средним между точностью (`pr`) и полнотой (`rec`).
    # Она определяется как \(F = \frac{2 \cdot \text{precision} \cdot \text{recall}}{\text{precision} + \text{recall}}\).
    # F-мера объединяет точность и полноту в единую метрику,
    # учитывая как правильность классификации положительных классов,
    # так и способность модели обнаруживать все объекты положительного класса.
    f_score = 2 * pr * rec / (pr + rec)

    # Возвращает вычисленное значение F-меры.
    return f_score

# Итак, функция `f_score` принимает на вход векторы истинных
# и предсказанных меток классов, вычисляет и возвращает F-меру модели.

F-score определяется как \(F = $\frac{2 \cdot \text{precision} \cdot \text{recall}}{\text{precision} + \text{recall}}$\).

In [67]:
f_score(y, pred)

0.888888888888889

In [68]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


# Выводит значение точности модели, которая вычисляется
# с помощью функции `accuracy_score` из библиотеки `scikit-learn`.
# Эта функция сравнивает векторы истинных меток `y`
# с предсказанными метками `pred` и вычисляет долю правильных предсказаний.
print(f'accuracy {accuracy_score(y, pred)}')

# Выводит значение точности модели, которая вычисляется
# с помощью функции `precision_score` из библиотеки `scikit-learn`.
# Эта функция вычисляет точность модели, которая характеризует
# способность модели классифицировать объекты положительного класса корректно.
print(f'precision {precision_score(y, pred)}')

# Выводит значение полноты модели, которая вычисляется
# с помощью функции `recall_score` из библиотеки `scikit-learn`.
# Эта функция вычисляет полноту модели, которая характеризует
# способность модели обнаруживать все объекты положительного класса.
print(f'recall {recall_score(y, pred)}')

# Выводит значение F-меры модели, которая вычисляется
# с помощью функции `f1_score` из библиотеки `scikit-learn`.
# F-мера является гармоническим средним между точностью и полнотой,
# и она характеризует сбалансированность модели между
# обнаружением объектов положительного класса и избежанием ложных срабатываний.
print(f'f-score {f1_score(y, pred)}')

# Выводит матрицу ошибок (confusion matrix), которая представляет собой
# таблицу, показывающую количество верно и ошибочно классифицированных наблюдений для каждого класса.
# Она вычисляется с помощью функции `confusion_matrix` из библиотеки `scikit-learn`.
print(f'\n {confusion_matrix(y, pred)}')

accuracy 0.9
precision 1.0
recall 0.8
f-score 0.8888888888888888

 [[5 0]
 [1 4]]
